In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

data_path = './data/'
df = pd.read_csv(data_path + 'train.csv', header=None)
df_label = pd.read_csv(data_path + 'trainLabels.csv', header=None)

In [2]:
print(df.shape)
df.head()

(1000, 40)


,0,1,2,3,4,5,6,7,8,9,...,30,31,32,33,34,35,36,37,38,39
0,0.299403,-1.226624,1.498425,-1.176150,5.289853,0.208297,2.404498,1.594506,-0.051608,0.663234,...,-0.850465,-0.622990,-1.833057,0.293024,3.552681,0.717611,3.305972,-2.715559,-2.682409,0.101050
1,-1.174176,0.332157,0.949919,-1.285328,2.199061,-0.151268,-0.427039,2.619246,-0.765884,-0.093780,...,-0.819750,0.012037,2.038836,0.468579,-0.517657,0.422326,0.803699,1.213219,1.382932,-1.817761
2,1.192222,-0.414371,0.067054,-2.233568,3.658881,0.089007,0.203439,-4.219054,-1.184919,-1.240310,...,-0.604501,0.750054,-3.360521,0.856988,-2.751451,-1.582735,1.672246,0.656438,-0.932473,2.987436
3,1.573270,-0.580318,-0.866332,-0.603812,3.125716,0.870321,-0.161992,4.499666,1.038741,-1.092716,...,1.022959,1.275598,-3.480110,-1.065252,2.153133,1.563539,2.767117,0.215748,0.619645,1.883397
4,-0.613071,-0.644204,1.112558,-0.032397,3.490142,-0.011935,1.443521,-4.290282,-1.761308,0.807652,...,0.513906,-1.803473,0.518579,-0.205029,-4.744566,-1.520015,1.830651,0.870772,-1.894609,0.408332


In [3]:
print(df_label.shape)
df_label = np.ravel(df_label)
print(df_label.shape)

(1000, 1)
(1000,)


In [4]:
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split, KFold, GridSearchCV

In [5]:
# logistic regression
from sklearn.linear_model import LogisticRegression
x_train, x_test, y_train, y_test = train_test_split(df, df_label, test_size=0.2, random_state=42)
logreg = LogisticRegression()

logreg.fit(x_train, y_train)

y_pred = logreg.predict(x_test)

In [6]:
print("Accuracy: ", metrics.accuracy_score(y_test, y_pred))

Accuracy:  0.825


In [7]:
# Random forest classifier
x_train, x_test, y_train, y_test = train_test_split(df, df_label, test_size=0.2, random_state=42)

clf = GradientBoostingClassifier(random_state=7)

clf.fit(x_train, y_train)

y_pred = clf.predict(x_test)

In [8]:
print("Accuracy: ", metrics.accuracy_score(y_test, y_pred))

Accuracy:  0.88


In [9]:
n_estimators = [200, 300, 400]
max_depth = [1, 3, 5, 10]
param_grid = dict(n_estimators=n_estimators, max_depth=max_depth)

grid_search = GridSearchCV(clf, param_grid, scoring='accuracy', n_jobs=-1, verbose=1)

grid_result = grid_search.fit(x_train, y_train)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   34.7s
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:   43.2s finished


In [10]:
print("Best Accuracy: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best Accuracy: 0.903750 using {'max_depth': 5, 'n_estimators': 300}


In [11]:
clf_bestparam = GradientBoostingClassifier(n_estimators = grid_result.best_params_['n_estimators'],
                                           max_depth = grid_result.best_params_['max_depth'])

clf_bestparam.fit(x_train, y_train)
y_pred = clf_bestparam.predict(x_test)

In [12]:
print("Accuracy: ", metrics.accuracy_score(y_test, y_pred))

Accuracy:  0.9


In [13]:
df_test = pd.read_csv(data_path + 'test.csv', header=None)
df_test.shape

(9000, 40)

In [14]:
y_pred_2 = clf_bestparam.predict(df_test)

In [15]:
# from sklearn.preprocessing import MinMaxScaler
# MMEncoder = MinMaxScaler()
# df = MMEncoder.fit_transform(df)
# df

array([[0.54768934, 0.3201633 , 0.8000649 , ..., 0.10885793, 0.38835347,
        0.54099147],
       [0.32748792, 0.54045601, 0.69543023, ..., 0.67281402, 0.68239271,
        0.40959366],
       [0.68110615, 0.43495393, 0.52701239, ..., 0.59289083, 0.51492339,
        0.73864753],
       ...,
       [0.63934342, 0.2484536 , 0.31448544, ..., 0.50749243, 0.64714023,
        0.58979183],
       [0.86753262, 0.38956923, 0.34207158, ..., 0.53164179, 0.41521043,
        0.29972255],
       [0.537168  , 0.48143774, 0.68144039, ..., 0.55613398, 0.3342069 ,
        0.69637177]])

In [16]:
# df = pd.DataFrame(df)
# df.head()

,0,1,2,3,4,5,6,7,8,9,...,30,31,32,33,34,35,36,37,38,39
0,0.547689,0.320163,0.800065,0.348793,0.638821,0.528876,0.649860,0.594141,0.491822,0.511760,...,0.406288,0.352617,0.400488,0.518257,0.813164,0.575869,0.726477,0.108858,0.388353,0.540991
1,0.327488,0.540456,0.695430,0.332123,0.547881,0.470356,0.458765,0.669273,0.378280,0.398251,...,0.411223,0.447978,0.658593,0.545890,0.503160,0.529535,0.518918,0.672814,0.682393,0.409594
2,0.681106,0.434954,0.527012,0.187336,0.590833,0.509461,0.501315,0.167901,0.311670,0.226338,...,0.445806,0.558805,0.298666,0.607026,0.333030,0.214913,0.590963,0.592891,0.514923,0.738648
3,0.738047,0.411502,0.348957,0.436184,0.575146,0.636623,0.476653,0.807142,0.665146,0.248469,...,0.707288,0.637725,0.290694,0.304463,0.706572,0.708607,0.681780,0.529632,0.627185,0.663044
4,0.411336,0.402473,0.726456,0.523433,0.585868,0.493033,0.585006,0.162679,0.220047,0.533414,...,0.625499,0.175345,0.557251,0.439863,0.181230,0.224755,0.604102,0.623657,0.445334,0.562034


In [17]:
# n_estimators = [200, 300, 400]
# max_depth = [1, 3, 5, 10]
# param_grid = dict(n_estimators=n_estimators, max_depth=max_depth)

# grid_search = GridSearchCV(clf, param_grid, scoring='accuracy', n_jobs=-1, verbose=1)

# grid_result = grid_search.fit(x_train, y_train)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   30.3s
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:   38.6s finished


In [18]:
# print("Best params: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best params: 0.903750 using {'max_depth': 5, 'n_estimators': 300}


In [19]:
# x_train, x_test, y_train, y_test = train_test_split(df, df_label, test_size=0.25, random_state=42)

# clf = GradientBoostingClassifier(n_estimators = grid_result.best_params_['n_estimators'],
#                                 max_depth = grid_result.best_params_['max_depth'])
# clf.fit(x_train, y_train)

# y_pred = clf.predict(x_test)

In [20]:
# print("Accuracy: ", metrics.accuracy_score(y_test, y_pred))

Accuracy:  0.892


In [21]:
Id = np.linspace(1, 9000, num=9000, dtype=int)
Id

array([   1,    2,    3, ..., 8998, 8999, 9000])

In [22]:
final_output = pd.DataFrame(Id, columns=['Id'])

In [23]:
final_output['Solution'] = y_pred_2

In [24]:
final_output.head()

,Id,Solution
0,1,1
1,2,0
2,3,0
3,4,0
4,5,0


In [25]:
final_output.to_csv('basic.csv', index=False)